# tfrec2gan

Tento skript připravuje data z původní adresářové struktury do formátu tfrecords
Původní adresářová strukttura má násladující formát:

G:\Data\hdr_data_new_labels>tree /f  
Folder PATH listing  
Volume serial number is 56C5-D1E5  
G:.  
├───exp3750  
│   ├───ob1  
│   │   │   processed_data.yaml  
│   │   │   robot_data.yaml  
│   │   │  
│   │   └───camera_imgs  
│   │           0000.jpg  
│   │           0001.jpg  
                ...  
│   │           0098.jpg  
│   │           0099.jpg  
│   │  
│   ├───ob10  
│   │   │   processed_data.yaml  
│   │   │   robot_data.yaml  
│   │   │  
│   │   └───camera_imgs  
│   │           0001_0.jpg  
│   │           0001_1.jpg  
│   │           0001_2.jpg  
│   │           0002_0.jpg  
│   │           0002_1.jpg  
                ...  
│   │           0019_1.jpg  
│   │           0019_2.jpg  
│   │           0020_0.jpg  
│   │           0020_1.jpg  
│   │           0020_2.jpg  
│   │  
│   ├───ob11  
│   │   │   processed_data.yaml  
│   │   │   robot_data.yaml  
│   │   │  
│   │   └───camera_imgs  
│   │           0001_0.jpg  
│   │           0001_1.jpg  
│   │           0001_2.jpg  
│   │           0002_0.jpg  
│   │           0002_1.jpg  
                ...  
│   ├───ob12  
        ...  
│   └───ob73  
│  
├───exp7500  
├───exp15000  
└───exp33000  

Kde adresáře exp3500, exp7500, exp15000 a exp3300 odpovídají jednotlivým expozicím.

Adresář pro každé expozice pak obsahuje adresáře jednotlivých objektů ob1, ..., ob73.
V adresáři objektu jsou dva yaml soubory, které popisují souřadnice úchopu a podadresář camera_imgs
s jednotlivými fotografiemi.


Zavedení potřebných modulů

In [12]:
import glob
import numpy as np
import tensorflow as tf
import os, re
import shutil
from PIL import Image

from tqdm import tqdm
import time

Definice třídy generátoru:

In [13]:
class GenerateTFRecord:
  def __init__(self):
    self.root = "G:\Data\hdr_data_new_labels"
    self.label = "\exp7500"
    #self.ob = "\ob1"
    self.ob = "\*"
    ob_train = [1, 5, 7, 8, 10, 11, 13, 14, 16, 17, 19, 20, 22, 23, 25, 26, 28, 29]
    ob_train += [31, 32, 34, 35, 37, 38, 40, 41, 43, 44, 46, 48, 50, 51, 53, 54, 56, 57, 59, 60, 62, 63, 65, 66, 71, 72]
    ob_test = [6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 49, 52, 55, 58, 61, 64, 67, 73]
    print(len(ob_train), len(ob_test), len(ob_train+ob_test))
    self.ob_dir = {'train': ob_train, 'test': ob_test, 'all': ob_train + ob_test}
    ob_test_exc = ["\\exp7500\\ob39\\camera_imgs\\0033_0.jpg", "\\exp7500\\ob39\\camera_imgs\\0037_0.jpg"]
    self.ob_exc = ob_test_exc
    #self.data = self.root + self.label + self.ob + "\processed_data.yaml"
    #print(self.data)

  def rexmatch(self, dir):
    rex = re.compile('^ob([1-9]|[1-9][0-9])$')
    if rex.match(dir):
      # print(dir, 'True')
      return True
    else:
      # print(dir, 'False')
      return False

  def wrtrec(self, imgpath, fpath):
    # write to tfrecord
    ximg = 2592
    yimg = 1944
    size = res, res

    img = Image.open(imgpath)
    img.thumbnail(size)
    #img = img.resize(size)
    npimg = np.array(img).astype('uint8')
    image = np.squeeze(npimg).flatten().tolist()
    #encfpath = [ord(char) for char in fpath]
    #encfpath.extend([32] * (50-len(encfpath)))
    #encfname = tf.compat.as_bytes(fname, encoding='utf-8')
    example = tf.train.Example(features=tf.train.Features(feature={
      #'fpath': tf.train.Feature(int64_list=tf.train.Int64List(value=encfpath)),
      'fpath': tf.train.Feature(bytes_list=tf.train.BytesList(value=[fpath.encode()])),
      'image': tf.train.Feature(int64_list=tf.train.Int64List(value=image)),
      'label': tf.train.Feature(float_list=tf.train.FloatList(value=self.coords))
    }))
    self.writer.write(example.SerializeToString())

  def create_tfrecord(self, tfrecord_file_name, res, dstype):
    listdir = self.ob_dir[dstype] #train/test switch
    self.numnames = len(listdir)
    pbar = tqdm(total=self.numnames);
    c = 0
    nf = 0
    Cimg = 0

    #print(tfrecord_file_name)
    with tf.io.TFRecordWriter(tfrecord_file_name) as self.writer:

      #for name in names:
      for ob in listdir:  # train/test switch
        nf += 1
        ob = 'ob' + str(ob)
        name = self.root + self.label + "\\" + ob + "\processed_data.yaml"
        #print(name)
        with open(name, 'r') as file:
          lines = file.readlines()
        #print(len(lines))
        #print(lines[0])

        i = 0; cimg = 0
        while i < len(lines):
          line = lines[i]

          if line.strip() == "- robot_to_object_tfm_descriptor:":
            self.coords = []
            for j in range(6):
              i = i + 1
              coord = lines[i].strip()[2:]
              self.coords.append(float(coord))
              # print(coord)
            i = i + 1
            fname = lines[i][12:].strip().replace("/", "\\")
            fpath = self.label + fname
            imgpath = self.root + self.label + fname
            # print(imgpath)
            #print(fpath)
            if fpath in self.ob_exc:
              print("EXC--->", fpath)
            else:
              cimg += 1
              self.wrtrec(imgpath, fpath)
          elif line[2:9] == "- fname":
            fname = line[12:].strip().replace("/", "\\")
            fpath = self.label + fname
            imgpath = self.root + self.label + fname
            # print(imgpath)
            i = i + 1  # robot_to_object_tfm_descriptor:
            self.coords = []
            for j in range(6):
              i = i + 1
              coord = lines[i].strip()[2:]
              self.coords.append(float(coord))
              # print(coord)
            #print(fpath)
            if fpath in self.ob_exc:
              print("EXC--->", fpath)
            else:
              cimg += 1
              self.wrtrec(imgpath, fpath)
            #print(fname)
          else:
            # read next line
            i = i + 1

        Cimg += cimg
        #if Cimg > 0.7 * 2962:
        #  break

        #pbar
        time.sleep(0.1)
        c = c + 1
        if c % 1 == 0:
          pbar.update(1)

    pbar.close()
    print(Cimg, nf)

Definice třídy extraktoru:

In [14]:
class TFRecordExtractor:
  def __init__(self, tfrecord_file, res):
    self.tfrecord_file = os.path.abspath(tfrecord_file)
    if res == 64:
      self.ximg = 64
      self.yimg = 48   #48 for img.thumbnail(size)
    elif res == 128:
      self.ximg = 128  #2592
      self.yimg = 96   #1944
    elif res == 256:
      self.ximg = 256
      self.yimg = 192

  def _extract_fn(self, tfrecord):
    # Extract features
    features = {
      # 'fpath': tf.io.FixedLenFeature([50], tf.int64),
      'fpath': tf.io.FixedLenFeature([1], tf.string),
      'image': tf.io.FixedLenFeature([self.ximg*self.yimg], tf.int64),
      'label': tf.io.FixedLenFeature([6], tf.float32)
    }

    # Extract the data record
    sample = tf.io.parse_single_example(tfrecord, features)
    fpath = sample['fpath']
    image = sample['image']
    label = sample['label']

    return [fpath, image, label]

  def extract_image(self):
    # Create folder to store extracted images
    folder_path = './ExtractedImages'
    shutil.rmtree(folder_path, ignore_errors=True)
    os.mkdir(folder_path)

    # Pipeline of dataset and iterator
    dataset = tf.data.TFRecordDataset([self.tfrecord_file])
    dataset = dataset.map(self._extract_fn)
    #iterator = dataset.make_one_shot_iterator()
    #next_element = iterator.get_next()

    # TF 2.0
    with open("coords.txt", 'w') as file:
      i = 3
      for next_element in dataset:
        #fpath = next_element[0]
        #fpath = ''.join(chr(i) for i in fpath).strip()
        fpath = next_element[0].numpy()[0].decode()
        image = next_element[1].numpy()
        coords = list(next_element[2].numpy())
        image = image.reshape(self.yimg, self.ximg)
        image = image.astype('uint8')
        save_path = os.path.abspath(os.path.join(folder_path, f"img_{i}.jpg"))
        image = Image.fromarray(image)
        image.save(save_path)
        print("Save path = ", save_path)
        #print("Image path = ", fpath)
        file.write(fpath + "\n")
        file.write(save_path+"\n")
        for coord in coords:
          line = str(coord)+"\n"
          file.write(line)
        file.write("---"+"\n")
        i = i + 1
        if i>=3+5:
          break

    return

In [15]:
if __name__ == '__main__':
    # Width resolution
    res = 64

    # Create
    t = GenerateTFRecord()
    dstype = 'all'
    t.create_tfrecord(f"gan_{res}{dstype}.tfrecord", res=res, dstype=dstype)

    # Extract
    dstype = 'all'
    t = TFRecordExtractor(f"gan_{res}{dstype}.tfrecord", res=res)
    t.extract_image()

  0%|                                                                                           | 0/66 [00:00<?, ?it/s]

44 22 66


 83%|████████████████████████████████████████████████████████████████████▎             | 55/66 [00:32<00:06,  1.70it/s]

EXC---> \exp7500\ob39\camera_imgs\0033_0.jpg
EXC---> \exp7500\ob39\camera_imgs\0037_0.jpg


100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [00:37<00:00,  1.78it/s]


2951 66
Save path =  E:\NCK\ntbs\ExtractedImages\img_3.jpg
Save path =  E:\NCK\ntbs\ExtractedImages\img_4.jpg
Save path =  E:\NCK\ntbs\ExtractedImages\img_5.jpg
Save path =  E:\NCK\ntbs\ExtractedImages\img_6.jpg
Save path =  E:\NCK\ntbs\ExtractedImages\img_7.jpg


<table><tr>
    <td><img src="ExtractedImages\img_3.jpg" alt="img_3" style="width: 64px;"/></td>
    <td><img src="ExtractedImages\img_4.jpg" alt="img_4" style="width: 64px;"/></td>
    <td><img src="ExtractedImages\img_5.jpg" alt="img_5" style="width: 64px;"/></td>
    <td><img src="ExtractedImages\img_6.jpg" alt="img_6" style="width: 64px;"/></td>
    <td><img src="ExtractedImages\img_7.jpg" alt="img_7" style="width: 64px;"/></td>
</tr></table>